# MCDC solver

In [ ]:
import re
import sys

In [ ]:
class Condition:
    def __init__(self):
        self.name = ''
        self.precop = ''
        self.trailop = ''

In [ ]:
class Testcase:
    def __init__(self, seq, res):
        self.seq = seq
        self.res = res

In [ ]:
class Decision:
    def __init__(self):
        self.conditions = list()
        self.testcases = list()
        
    def extract(self, raw):
        elem = list(raw)
        size = len(raw)
        i = 0
        while i < size:
            temp = Condition()
            temp.name = elem[i]
            if i == 0:
                temp.precop = '-'
            else:
                temp.precop = elem[i-1]
            if (i+1) == size:
                temp.trailop = '-'
            else:
                temp.trailop = elem[i+1]
            self.conditions.append(temp)
            i += 2
    
    def derivation(self):
        testcase = list()
        length = len(self.conditions)
        self.testcases = [None] * 2 * length
        for i in range(0, length):
            for x in range(0, length - (length - i)):
                if self.conditions[x].trailop == "&":
                    testcase.append("1")
                else:
                    testcase.append("0")
            testcase.append("1")
            for x in range(i+1, length):
                if self.conditions[x].precop == "&":
                    testcase.append("1")
                else:
                    testcase.append("0")
            self.testcases[i] = Testcase(''.join(testcase), "True")
            testcase[i]=("0")
            self.testcases[i + length] = Testcase(''.join(testcase), "False")
            testcase.clear()
    
    def reduce(self):
        li = list()
        temp = list()
        for testcase in self.testcases:
            if testcase.seq not in li:
                li.append(testcase.seq)
                temp.append(testcase)
        self.testcases.clear()
        self.testcases = temp
        
    def print_testcases(self):
        i = 1
        for x in self.testcases:
            print('Testcase ' + str(i) + ': ' + x.seq + '   ' + x.res)
            i += 1

In [ ]:
class DecisionError(Exception):
    def __init__(self, value):
        self.value = value
    def __str__(self):
        return repr(self.value)

In [ ]:
def solve_decision(sequence):
    try:
        if re.match(r"^[A-Za-z]{1}((\||\&)[A-Za-z])*$", sequence) == None: 
            raise DecisionError('Wrong layout')
        seq_split = re.split("\||\&", sequence)
        if len(seq_split) != len(set(seq_split)):
            raise DecisionError('Duplicate letters')
        
    except DecisionError as e:
        if e.value == 'Wrong layout':
            print('Decision has to start with a letter and can be followed by combinations of a "|" or "&" and letter. Got: ', sequence)
        else:
            print('A letter can only be used once within a decision')
        raise
    decision = Decision()
    decision.extract(sequence)
    decision.derivation()
    decision.reduce()
    decision.print_testcases()


In [ ]:
# Give here the decision you want to analyse
# The decision should contain at least one variable represented by a letter.
# The decision can contain a letter only onces
# The first letter can be followed by a combination of '|' or '&' and a letter.
# Example: 'A&B|C'

decision = ''

solve_decision(decision)